In [ ]:
import arrow
import socket
from sqlalchemy.orm import Session
from tqdm import tqdm

from april import Evaluator
from april.anomalydetection import *
from april.database import EventLog
from april.database import Model
from april.database import get_engine
from april.dataset import Dataset
from april.fs import DATE_FORMAT
from april.fs import get_event_log_files
from pm4py.objects.log.importer.xes import importer as xes_importer
import pandas as pd
import pickle

In [ ]:
def common_member(a, b):   
    a_set = set(a)
    b_set = set(b)     
    # check length
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set)) 
    else:
        return("no common elements")

In [ ]:
def fit_and_save(dataset_name, ad, ad_kwargs=None, fit_kwargs=None):
    if ad_kwargs is None:
        ad_kwargs = {}
    if fit_kwargs is None:
        fit_kwargs = {}

    # Save start time
    start_time = arrow.now()

    # Dataset
    dataset = Dataset(dataset_name)

    # AD
    ad = ad(**ad_kwargs)

    # Train and save
    ad.fit(dataset, **fit_kwargs)
    file_name = f'{dataset_name}_{ad.abbreviation}_{start_time.format(DATE_FORMAT)}'
    model_file = ad.save(file_name)

    # Save end time
    end_time = arrow.now()
    return ad

In [ ]:
def threshold_att (event_data, final_scores, alpha):
    number_cases = len(event_data)
    num_events = 0
    for i in range (number_cases):
        num_events += len(event_data[i])
    temp = np.sum(final_scores, axis=0)
    threshold_arr = np.sum(temp, axis=0)
    threshold_arr = alpha*threshold_arr/num_events
    return threshold_arr

In [ ]:
datasets = sorted([e.name for e in get_event_log_files() if e.p == 0.3])
ads = [dict(ad=BINetv2, fit_kwargs=dict(epochs=50, batch_size=100))]
for ad in ads:
    model = [fit_and_save(d, **ad) for d in tqdm(datasets, desc=ad['ad'].name)]

In [ ]:
datasets = sorted([e.name for e in get_event_log_files()])
ads = [dict(ad=BINetv2, fit_kwargs=dict(epochs=50, batch_size=100))]
for ad in ads:
    model = [fit_and_save(d, **ad) for d in tqdm(datasets, desc=ad['ad'].name)]

In [ ]:
print(model)
p1 = Evaluator(r'C:\Users\ASUS\.out\models\wide-0.1-1_binetv2_20220711-231436.646890.model').result
print(dir(Evaluator))
print(p1)
final_scores = p1.scores
print(final_scores.shape)
print(type(final_scores))

In [ ]:
event_log_path = r'D:\Final master thesis evaluation\datasets\5\wide-0.1-1.xes'
event_data = xes_importer.apply(event_log_path)

In [ ]:
label_csv = pd.read_csv(r'D:\Final master thesis evaluation\wide-five\wide-0.1-1.csv')
loss_df_path  = r'D:\Final master thesis evaluation\wide-five\binet_loss.pt'
label_csv.drop("Unnamed: 0", axis=1, inplace=True)
label_csv["case_id"] = pd.to_numeric(label_csv["case_id"])
label_csv.case_id.astype(str).astype(int)
label_csv.label.astype(str)
test_csv = label_csv.loc[label_csv['case_id'] > 12000]
label_csv1 = test_csv.loc[test_csv['label']!= 'normal']
M = list(label_csv1['case_id'])
print(len(test_csv))
print(len(label_csv1))
#print(M)